In [ ]:
import os
import torch
import numpy as np
import mmcv
from mmengine.config import Config
from mmengine.runner import Runner
from mmengine.registry import DATASETS
from mmdet.utils import register_all_modules
from mmdet.apis import inference_detector, init_detector
from mmdet.visualization import DetLocalVisualizer
from mmengine.logging import HistoryBuffer

# ✅ Paksa torch.load agar selalu pakai weights_only=False
_orig_torch_load = torch.load
def torch_load_wrapper(*args, **kwargs):
    kwargs["weights_only"] = False  # override default baru di PyTorch 2.6
    return _orig_torch_load(*args, **kwargs)

torch.load = torch_load_wrapper

# ========== KONFIG DASAR ==========
DEBUG_MODE = False  # True untuk debug cepat
print("="*50)
print(f"      MODE DEBUGGING: {'AKTIF' if DEBUG_MODE else 'NONAKTIF'}")
print("="*50)

register_all_modules()

print("--- Tes Ketersediaan GPU PyTorch ---")
print(f"Apakah CUDA tersedia? -> {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU terdeteksi: {torch.cuda.get_device_name(0)}")
else:
    print("PERINGATAN: Tidak ada GPU. Training di CPU akan lambat.")

      MODE DEBUGGING: NONAKTIF
--- Tes Ketersediaan GPU PyTorch ---
Apakah CUDA tersedia? -> True
GPU terdeteksi: NVIDIA GeForce RTX 5060


In [ ]:
# ===== Step 1: Load Config RetinaNet + Swin Backbone =====
print("--- INFO: Memilih model backbone Swin Transformer ---")
# cfg_path = 'mmdetection/configs/swin/retinanet_swin-t-p4-w7_fpn_1x_coco.py'
cfg_path = 'mmdetection/configs/swin/retinanet_swin-s-p4-w7_fpn_1x_coco_custom.py'
cfg = Config.fromfile(cfg_path)

# Nama model cuma untuk penamaan work_dir
model_name = "retinanet" if "retinanet" in cfg_path.lower() else "custommodel"
print(f"--- INFO: Model yang digunakan: {model_name} ---")

--- INFO: Memilih model backbone Swin Transformer ---
--- INFO: Model yang digunakan: retinanet ---


In [ ]:
# ===== Step 2: Definisikan Pipeline Dasar =====
print("--- INFO: Mendefinisikan pipeline dasar untuk training dan validasi ---")

# Pipeline ini akan diterapkan pada SETIAP gambar SEBELUM digabungkan oleh Mosaic.
# Ini adalah pipeline "mentah" untuk memuat dan me-resize gambar.
train_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=False),
    dict(type='Resize', scale=(640, 640), keep_ratio=True),
    # Kita menggunakan Pad untuk memastikan semua gambar memiliki ukuran yang sama persis
    # Ini penting agar Mosaic dapat menggabungkannya tanpa error.
    dict(type='Pad', size=(640, 640), pad_val=dict(img=(114, 114, 114))),
]

# Pipeline validasi tidak berubah dan tidak menggunakan Mosaic.
val_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(type='Resize', scale=(640, 640), keep_ratio=True),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=False),
    dict(type='PackDetInputs',
         meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape', 'scale_factor'))
]

print("--- INFO: Definisi pipeline selesai. Dataset akan dibangun di Step 3. ---")

--- INFO: Mendefinisikan pipeline dasar untuk training dan validasi ---
--- INFO: Definisi pipeline selesai. Dataset akan dibangun di Step 3. ---


In [ ]:
# ===== Step 3: Dataset & Kelas (Dengan Wrapper Mosaic) =====

# 1. Definisikan 10 kelas yang ingin Anda gunakan. Urutan di sini akan menentukan label model (0-9).
CLASSES = (
    'Betta', 'Discuss', 'Glofish', 'Goldfish', 'Guppy', 'Gurami',
    'Manfish', 'Molly', 'Swordtail', 'Tiger Barb'
)

# 2. Buat pemetaan dari ID asli di file JSON ke label baru (0-9).
# PENTING: Kita sengaja TIDAK menyertakan ID 2 (Corydoras) dan ID 8 (Janitor).
cat_id_map = {
    1: 0,  # Betta
    3: 1,  # Discuss
    4: 2,  # Glofish
    5: 3,  # Goldfish
    6: 4,  # Guppy
    7: 5,  # Gurami
    9: 6,  # Manfish
    10: 7, # Molly
    11: 8, # Swordtail
    12: 9  # Tiger Barb
}

# 3. Palet warna Anda yang berisi 10 warna sekarang sudah pas.
PALETTE = [
    (220, 20, 60), (0, 128, 255), (60, 179, 113), (255, 140, 0), (138, 43, 226),
    (255, 215, 0), (199, 21, 133), (127, 255, 212), (70, 130, 180), (255, 99, 71)
]

# Metainfo akan secara otomatis menggunakan variabel yang sudah diperbarui di atas.
metainfo = {'classes': CLASSES, 'palette': PALETTE, 'cat2label': cat_id_map}

print(f"--- INFO: Konfigurasi diatur untuk melatih {len(CLASSES)} kelas, mengabaikan kelas lain dari dataset. ---")

data_root = 'dataset/10_jenis_ikan/'
train_ann_file = 'train/_annotations.coco.json' if DEBUG_MODE else 'train/_annotations.coco.json'
val_ann_file   = 'valid/_annotations.coco.json' if DEBUG_MODE else 'valid/_annotations.coco.json'
print(f"--- INFO: Anotasi train: {train_ann_file}")
print(f"--- INFO: Anotasi val  : {val_ann_file}")


# --- BAGIAN UTAMA: MEMBANGUN KONFIGURASI DATASET ---

# 1. Definisikan dataset dasar untuk training.
train_dataset_cfg = dict(
    type='CocoDataset',
    data_root=data_root,
    metainfo=metainfo,
    ann_file=train_ann_file,
    data_prefix=dict(img='train/'),
    filter_cfg=dict(filter_empty_gt=True),
    # Pipeline ini diterapkan pada gambar individual SEBELUM mosaic.
    pipeline=train_pipeline 
)

# 2. Bungkus dataset training dengan wrapper untuk Mosaic.
# Pipeline di dalam wrapper ini diterapkan SETELAH gambar digabung.
cfg.train_dataloader.dataset = dict(
    type='MultiImageMixDataset',
    dataset=train_dataset_cfg,
    pipeline=[
        dict(type='PhotoMetricDistortion'),
        dict(type='RandomFlip', prob=0.5), # Flip bisa di sini (untuk seluruh mosaic) atau di pipeline dasar
        dict(type='Normalize',
             mean=[123.675, 116.28, 103.53],
             std=[58.395, 57.12, 57.375],
             to_rgb=True),
        dict(type='PackDetInputs')
    ]
)

# 3. Bangun dataset validasi dan test seperti biasa.
# PENTING: Jangan lupa untuk menyertakan 'pipeline=val_pipeline' di sini.
cfg.val_dataloader.dataset.update(dict(
    data_root=data_root, metainfo=metainfo, ann_file=val_ann_file,
    data_prefix=dict(img='valid/'), filter_cfg=dict(filter_empty_gt=True),
    pipeline=val_pipeline
))
cfg.test_dataloader.dataset.update(dict(
    data_root=data_root, metainfo=metainfo, ann_file=val_ann_file,
    # Untuk test set, kita biasanya tidak memfilter gambar tanpa anotasi
    data_prefix=dict(img='valid/'), filter_cfg=dict(filter_empty_gt=False), 
    pipeline=val_pipeline
))

--- INFO: Anotasi train: train/_annotations.coco.json
--- INFO: Anotasi val  : valid/_annotations.coco.json


In [ ]:
# ===== Step 4: Evaluator COCO (bbox) + classwise =====
cfg.val_evaluator.ann_file = os.path.join(data_root, val_ann_file)
cfg.test_evaluator.ann_file = os.path.join(data_root, val_ann_file)
cfg.val_evaluator.metric = 'bbox'
cfg.test_evaluator.metric = 'bbox'
# tampilkan AP per kelas + simpan hasil JSON prediksi untuk analisis lanjut
cfg.val_evaluator.classwise = True
cfg.test_evaluator.classwise = True
cfg.val_evaluator.outfile_prefix = os.path.join('work_eval', 'val_results')
cfg.test_evaluator.outfile_prefix = os.path.join('work_eval', 'test_results')
os.makedirs('work_eval', exist_ok=True)

In [ ]:
# ===== Step 5: Atur jumlah kelas di head (10 kelas) =====
if hasattr(cfg.model, 'bbox_head'):
    cfg.model.bbox_head.num_classes = len(CLASSES)
elif hasattr(cfg.model, 'roi_head'):
    if isinstance(cfg.model.roi_head.bbox_head, list):
        for head in cfg.model.roi_head.bbox_head:
            head.num_classes = len(CLASSES)
    else:
        cfg.model.roi_head.bbox_head.num_classes = len(CLASSES)

In [ ]:
# ===== Step 6: Training Settings - FINE-TUNING DENGAN MOSAIC =====

# --- BAGIAN 1: DEFINISI LEARNING RATE SCHEDULER (SUDAH DIPERBAIKI) ---
cfg.param_scheduler = [
    dict(
        type='LinearLR',
        start_factor=0.001,
        by_epoch=False,
        begin=0,
        end=500),
    dict(
        type='CosineAnnealingLR',
        by_epoch=True,
        begin=1,
        # DIUBAH: Langsung definisikan dengan nilai yang benar
        T_max=95, 
        eta_min_ratio=1e-2)
]
print("--- INFO: Menggunakan Learning Rate Scheduler (Warmup + CosineAnnealing) ---")


# --- BAGIAN 2: PENGATURAN PEMUATAN BOBOT ---
if not DEBUG_MODE:
    # Ganti dengan path checkpoint TERBAIK dari Eksperimen #5
    best_checkpoint_path = './outputs_swin_retinanet_10fish_finetune_color/best_coco_bbox_mAP_epoch_44.pth'
    
    if os.path.exists(best_checkpoint_path):
        cfg.load_from = best_checkpoint_path
        cfg.resume = False
        print(f"--- INFO: Memulai fine-tuning dengan MOSAIC, memuat bobot dari: {cfg.load_from} ---")
    else:
        cfg.load_from = None
        cfg.resume = False
        print(f"--- PERINGATAN: Checkpoint terbaik tidak ditemukan di '{best_checkpoint_path}'. Memulai dari awal. ---")

# --- BAGIAN 3: PENGATURAN EPOCH & HOOKS ---
if DEBUG_MODE:
    cfg.train_cfg.max_epochs = 2
    cfg.default_hooks.logger.interval = 5
    cfg.default_hooks.checkpoint.interval = 1
else:
    # Naikkan epoch karena Mosaic butuh waktu lebih lama untuk dipelajari
    cfg.train_cfg.max_epochs = 95
    cfg.default_hooks.logger.interval = 10
    cfg.default_hooks.checkpoint.interval = 1

# Pengaturan hook lainnya tetap sama.
cfg.default_hooks.checkpoint.max_keep_ckpts = 3
cfg.default_hooks.checkpoint.save_best = 'coco/bbox_mAP'
cfg.default_hooks.checkpoint.rule = 'greater'
cfg.visualizer.vis_backends = [
    dict(type='LocalVisBackend'),
    dict(type='TensorboardVisBackend')
]

print(f"--- INFO: Training akan berjalan selama {cfg.train_cfg.max_epochs} epoch. ---")

--- INFO: Menggunakan Learning Rate Scheduler (Warmup + CosineAnnealing) ---
--- INFO: Memulai fine-tuning dengan MOSAIC, memuat bobot dari: ./outputs_swin_retinanet_10fish_finetune_color/best_coco_bbox_mAP_epoch_44.pth ---
--- INFO: Training akan berjalan selama 95 epoch. ---


In [ ]:
# ===== Step 7: Work dir =====
work_dir_base = f'./outputs_swin_retinanet_10fish_finetune_mosaic' # Nama baru yang deskriptif
cfg.work_dir = f"{work_dir_base}_debug" if DEBUG_MODE else work_dir_base
print(f"--- INFO: Output ke: {cfg.work_dir} ---")
os.makedirs(cfg.work_dir, exist_ok=True)

--- INFO: Output ke: ./outputs_swin_retinanet_10fish_finetune_mosaic ---


In [9]:
# ===== Step 8: Sanity check dataset =====
print("\n--- Pengecekan Dataset Final ---")
try:
    train_dataset = DATASETS.build(cfg.train_dataloader.dataset)
    print(f"Jumlah data training: {len(train_dataset)}")
    val_dataset = DATASETS.build(cfg.val_dataloader.dataset)
    print(f"Jumlah data validasi: {len(val_dataset)}")
except Exception as e:
    print(f"\nFATAL ERROR memuat dataset: {e}")
    raise e
print("---------------------------------\n")

print("Konfigurasi selesai. Siap training.")


--- Pengecekan Dataset Final ---
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Jumlah data training: 1600
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Jumlah data validasi: 444
---------------------------------

Konfigurasi selesai. Siap training.


In [10]:
# ===== Step 9: Training =====
print(">>> Mulai training...")
runner = Runner.from_cfg(cfg)
runner.train()
print(">>> Training selesai.")

>>> Mulai training...
09/06 12:13:03 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: win32
    Python: 3.10.18 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:08:55) [MSC v.1929 64 bit (AMD64)]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 468793524
    GPU 0: NVIDIA GeForce RTX 5060
    CUDA_HOME: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8
    NVCC: Cuda compilation tools, release 12.8, V12.8.61
    MSVC: n/a, reason: fileno
    PyTorch: 2.8.0+cu128
    PyTorch compiling details: PyTorch built with:
  - C++ Version: 201703
  - MSVC 193833145
  - Intel(R) oneAPI Math Kernel Library Version 2025.2-Product Build 20250620 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.7.1 (Git Hash 8d263e693366ef8db40acc569cc7d8edf644556d)
  - OpenMP 2019
  - LAPACK is enabled (usually provided by MKL)
  - CPU capability usage: AVX2
  - CUDA Runtime 12.8
  - NVCC archi

In [11]:
# ========== STEP 10: EVALUATION & REPORTING ==========
# Evaluasi COCO mAP + per-class AP
print(">>> Evaluasi (COCO mAP + class-wise)...")
eval_results = runner.test()  # pakai cfg.test_dataloader + test_evaluator

if isinstance(eval_results, (list, tuple)) and len(eval_results) > 0:
    eval_dict = eval_results[0]
else:
    eval_dict = eval_results

# Simpan ringkasan evaluasi ke file teks
report_dir = os.path.join(cfg.work_dir, 'reports')
os.makedirs(report_dir, exist_ok=True)
eval_txt = os.path.join(report_dir, 'coco_eval_summary.txt')
with open(eval_txt, 'w', encoding='utf-8') as f:
    f.write("=== COCO BBox Evaluation (mAP) ===\n")
    for k, v in eval_dict.items():
        f.write(f"{k}: {v}\n")
print(f">>> Ringkasan evaluasi disimpan di: {eval_txt}")

# ========== Evaluator Custom: Confusion Matrix & PR Curve ==========
import numpy as np
import matplotlib.pyplot as plt
import itertools
from mmdet.apis import inference_detector, init_detector

def compute_iou_matrix(boxes1, boxes2):
    if boxes1.size == 0 or boxes2.size == 0:
        return np.zeros((boxes1.shape[0], boxes2.shape[0]), dtype=np.float32)
    x11, y11, x12, y12 = boxes1[:,0], boxes1[:,1], boxes1[:,2], boxes1[:,3]
    x21, y21, x22, y22 = boxes2[:,0], boxes2[:,1], boxes2[:,2], boxes2[:,3]
    inter_x1 = np.maximum(x11[:, None], x21[None, :])
    inter_y1 = np.maximum(y11[:, None], y21[None, :])
    inter_x2 = np.minimum(x12[:, None], x22[None, :])
    inter_y2 = np.minimum(y12[:, None], y22[None, :])
    inter_w = np.maximum(0, inter_x2 - inter_x1)
    inter_h = np.maximum(0, inter_y2 - inter_y1)
    inter = inter_w * inter_h
    area1 = (x12 - x11) * (y12 - y11)
    area2 = (x22 - x21) * (y22 - y21)
    union = area1[:, None] + area2[None, :] - inter
    iou = np.where(union > 0, inter / union, 0.0)
    return iou

def greedy_match(iou_mat, iou_thr=0.5):
    matches = []
    if iou_mat.size == 0:
        return matches
    gt_used, pred_used = set(), set()
    pairs = [(i, j, iou_mat[i, j]) for i in range(iou_mat.shape[0]) for j in range(iou_mat.shape[1])]
    pairs.sort(key=lambda x: x[2], reverse=True)
    for i, j, iou in pairs:
        if iou < iou_thr: 
            break
        if i in gt_used or j in pred_used:
            continue
        gt_used.add(i); pred_used.add(j)
        matches.append((i, j))
    return matches

def evaluate_confusion_pr(model, dataset, score_thr=0.3, iou_thr=0.5):
    # Ambil dataset asli (kalau dibungkus Concat/Repeat)
    base_dataset = dataset.dataset if hasattr(dataset, 'dataset') else dataset

    n_classes = len(CLASSES)
    conf_mat = np.zeros((n_classes, n_classes), dtype=np.int32)
    per_class_counts = {'TP': np.zeros(n_classes, dtype=np.int32),
                        'FP': np.zeros(n_classes, dtype=np.int32),
                        'FN': np.zeros(n_classes, dtype=np.int32)}
    pr_store = {c: {'scores': [], 'match': []} for c in range(n_classes)}

    for idx in range(len(base_dataset)):
        info = base_dataset.get_data_info(idx)
        img_path = info['img_path']

        # ground truth
        if 'instances' in info:
            gt_bboxes = np.array([ann['bbox'] for ann in info['instances']], dtype=np.float32).reshape(-1, 4)
            gt_labels = np.array([ann['bbox_label'] for ann in info['instances']], dtype=np.int64)
        else:
            gt_bboxes = np.zeros((0, 4), dtype=np.float32)
            gt_labels = np.array([], dtype=np.int64)

        # prediksi
        result = inference_detector(model, img_path)
        pred = result.pred_instances
        pred_bboxes = pred.bboxes.cpu().numpy()
        pred_scores = pred.scores.cpu().numpy()
        pred_labels = pred.labels.cpu().numpy()

        keep = pred_scores >= score_thr
        pred_bboxes, pred_scores_k, pred_labels_k = pred_bboxes[keep], pred_scores[keep], pred_labels[keep]

        # matching
        iou_mat = compute_iou_matrix(gt_bboxes, pred_bboxes)
        matches = greedy_match(iou_mat, iou_thr=iou_thr)
        matched_gt = {m[0] for m in matches}
        matched_pred = {m[1] for m in matches}

        # isi confusion
        for gi, pj in matches:
            gt_c, pd_c = int(gt_labels[gi]), int(pred_labels_k[pj])
            conf_mat[gt_c, pd_c] += 1
            per_class_counts['TP'][pd_c] += 1
            pr_store[pd_c]['scores'].append(float(pred_scores_k[pj]))
            pr_store[pd_c]['match'].append(1 if gt_c == pd_c else 0)

        # FP
        for j in range(len(pred_bboxes)):
            if j not in matched_pred:
                pd_c = int(pred_labels_k[j])
                per_class_counts['FP'][pd_c] += 1
                pr_store[pd_c]['scores'].append(float(pred_scores_k[j]))
                pr_store[pd_c]['match'].append(0)

        # FN
        for i in range(len(gt_bboxes)):
            if i not in matched_gt:
                gt_c = int(gt_labels[i])
                per_class_counts['FN'][gt_c] += 1

    # precision recall f1
    eps = 1e-12
    prec = per_class_counts['TP'] / (per_class_counts['TP'] + per_class_counts['FP'] + eps)
    rec  = per_class_counts['TP'] / (per_class_counts['TP'] + per_class_counts['FN'] + eps)
    f1   = 2 * prec * rec / (prec + rec + eps)

    # laporan
    lines = ["KELAS, TP, FP, FN, Precision, Recall, F1"]
    for c, name in enumerate(CLASSES):
        lines.append(f"{name}, {per_class_counts['TP'][c]}, {per_class_counts['FP'][c]}, {per_class_counts['FN'][c]}, "
                     f"{prec[c]:.4f}, {rec[c]:.4f}, {f1[c]:.4f}")

    per_class_txt = os.path.join(report_dir, "per_class_metrics.txt")
    with open(per_class_txt, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))

    # confusion matrix plot
    fig_cm = plt.figure(figsize=(10, 8))
    ax = fig_cm.add_subplot(111)
    im = ax.imshow(conf_mat, interpolation='nearest')
    ax.set_title("Confusion Matrix (IoU>0.5)")
    plt.colorbar(im, ax=ax)
    tick_marks = np.arange(len(CLASSES))
    ax.set_xticks(tick_marks, labels=CLASSES, rotation=45, ha='right')
    ax.set_yticks(tick_marks, labels=CLASSES)
    thresh = conf_mat.max() * 0.6 if conf_mat.max()>0 else 0.5
    for i, j in itertools.product(range(conf_mat.shape[0]), range(conf_mat.shape[1])):
        ax.text(j, i, conf_mat[i, j], ha="center", va="center",
                color="white" if conf_mat[i, j] > thresh else "black", fontsize=8)
    plt.tight_layout()
    cm_path = os.path.join(report_dir, "confusion_matrix.png")
    fig_cm.savefig(cm_path, dpi=200)
    plt.close(fig_cm)

    # PR curve per class
    for c in range(len(CLASSES)):
        scores = np.array(pr_store[c]['scores'])
        match  = np.array(pr_store[c]['match'])
        if scores.size == 0:
            continue
        order = np.argsort(-scores)
        scores, match = scores[order], match[order]
        tp_cum = np.cumsum(match)
        fp_cum = np.cumsum(1 - match)
        precision = tp_cum / np.maximum(tp_cum + fp_cum, 1)
        total_pos = per_class_counts['TP'][c] + per_class_counts['FN'][c]
        recall = tp_cum / max(total_pos, 1)
        fig_pr = plt.figure()
        plt.plot(recall, precision)
        plt.xlabel("Recall"); plt.ylabel("Precision")
        plt.title(f"PR Curve - {CLASSES[c]}")
        pr_path = os.path.join(report_dir, f"pr_curve_{c:02d}_{CLASSES[c].replace(' ','_')}.png")
        fig_pr.savefig(pr_path, dpi=200, bbox_inches='tight')
        plt.close(fig_pr)

    print(f">>> Confusion matrix: {cm_path}")
    print(f">>> Per-class metrics: {per_class_txt}")
    print(f">>> PR curves disimpan di: {report_dir}")

    return {'confusion_matrix': conf_mat,
            'per_class': {'precision': prec, 'recall': rec, 'f1': f1},
            'paths': {'cm': cm_path, 'txt': per_class_txt, 'dir': report_dir}}

# ========== Load Model untuk Evaluasi ==========
best_checkpoint_name = None
if os.path.exists(cfg.work_dir):
    best_ckpts = [f for f in os.listdir(cfg.work_dir) if f.startswith('best_') and f.endswith('.pth')]
    if best_ckpts:
        best_checkpoint_name = best_ckpts[0]
checkpoint_path = os.path.join(cfg.work_dir, best_checkpoint_name) if best_checkpoint_name else os.path.join(cfg.work_dir, "latest.pth")
print(f"--- INFO: Menggunakan checkpoint: {checkpoint_path} ---")

orig_load = torch.load
def torch_load_wrapper(*args, **kwargs):
    kwargs["weights_only"] = False
    return orig_load(*args, **kwargs)
torch.load = torch_load_wrapper

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = init_detector(cfg, checkpoint_path, device=device)

# Jalankan evaluator custom
_ = evaluate_confusion_pr(model, val_dataset, score_thr=0.3, iou_thr=0.5)


>>> Evaluasi (COCO mAP + class-wise)...
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "c:\Users\Stevenstven\anaconda3\envs\ikan_env\lib\site-packages\torch\utils\data\_utils\worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "c:\Users\Stevenstven\anaconda3\envs\ikan_env\lib\site-packages\torch\utils\data\_utils\fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "c:\Users\Stevenstven\anaconda3\envs\ikan_env\lib\site-packages\torch\utils\data\_utils\fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "c:\Users\Stevenstven\anaconda3\envs\ikan_env\lib\site-packages\mmengine\dataset\base_dataset.py", line 403, in __getitem__
    data = self.prepare_data(idx)
  File "c:\Users\Stevenstven\anaconda3\envs\ikan_env\lib\site-packages\mmengine\dataset\base_dataset.py", line 793, in prepare_data

In [ ]:
# ========== STEP 11: INFERENCE + RINGKASAN JUMLAH PER JENIS ==========
vis_save_dir = os.path.join(cfg.work_dir, 'vis_outputs')
os.makedirs(vis_save_dir, exist_ok=True)
visualizer = DetLocalVisualizer(
    vis_backends=[dict(type='LocalVisBackend')],
    name='visualizer',
    save_dir=vis_save_dir
)
visualizer.dataset_meta = cfg.train_dataloader.dataset.metainfo

img_path = 'dataset/10jenisikantest/test/Manfish_120_jpg.rf.d6bdd77a6bf451129c68ddd311776eb4.jpg'  # ganti gambar uji kamu
if not os.path.exists(img_path):
    print(f"ERROR: Gambar uji tidak ditemukan di '{img_path}'")
else:
    result = inference_detector(model, img_path)
    pred_instances = result.pred_instances
    scores = pred_instances.scores.cpu().numpy()
    labels = pred_instances.labels.cpu().numpy()

    score_thr = 0.3
    keep = scores >= score_thr
    labels_k = labels[keep]
    counts = {name: int((labels_k == i).sum()) for i, name in enumerate(CLASSES)}
    total_detected = int(keep.sum())

    # Tampilkan ringkasan di console
    print(">>> Summary Deteksi per Jenis (score_thr=0.3):")
    for k, v in counts.items():
        print(f"- {k}: {v}")
    print(f"Total semua ikan terdeteksi: {total_detected}")

    # Simpan ringkasan ke file
    summary_txt = os.path.join(vis_save_dir, "summary_counts.txt")
    with open(summary_txt, "w", encoding="utf-8") as f:
        f.write("Summary Deteksi per Jenis (score_thr=0.3)\n")
        for k, v in counts.items():
            f.write(f"{k}: {v}\n")
        f.write(f"TOTAL: {total_detected}\n")
    print(f">>> Ringkasan disimpan: {summary_txt}")

    # Visualisasi hasil ke file
    img = mmcv.imread(img_path)
    img = mmcv.imconvert(img, 'bgr', 'rgb')
    out_file = os.path.join(vis_save_dir, "deteksi_result.jpg")
    visualizer.add_datasample(
        name='result',
        image=img,
        data_sample=result,
        draw_gt=False,
        show=False,           # set True bila ingin popup
        wait_time=0,
        pred_score_thr=score_thr,
        out_file=out_file
    )
    print(f">>> Gambar hasil deteksi disimpan di: {out_file}")

In [ ]:
import os
import torch
import numpy as np
import mmcv
from mmengine.config import Config
from mmengine.runner import Runner
from mmengine.registry import DATASETS
from mmdet.utils import register_all_modules
from mmdet.apis import inference_detector, init_detector
from mmdet.visualization import DetLocalVisualizer
from mmengine.logging import HistoryBuffer
import matplotlib.pyplot as plt
import itertools

# ✅ Paksa torch.load agar selalu pakai weights_only=False
_orig_torch_load = torch.load
def torch_load_wrapper(*args, **kwargs):
    kwargs["weights_only"] = False
    return _orig_torch_load(*args, **kwargs)
torch.load = torch_load_wrapper

# ========== KONFIG DASAR ==========
DEBUG_MODE = False
print("="*50)
print(f"      MODE DEBUGGING: {'AKTIF' if DEBUG_MODE else 'NONAKTIF'}")
print("="*50)

register_all_modules()

print("--- Tes Ketersediaan GPU PyTorch ---")
print(f"Apakah CUDA tersedia? -> {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU terdeteksi: {torch.cuda.get_device_name(0)}")
else:
    print("PERINGATAN: Tidak ada GPU. Training di CPU akan lambat.")

# ===== Step 1: Load Config RetinaNet + Swin Backbone =====
print("--- INFO: Memilih model backbone Swin Transformer ---")
cfg_path = 'mmdetection/configs/swin/retinanet_swin-s-p4-w7_fpn_1x_coco_custom.py'
cfg = Config.fromfile(cfg_path)
model_name = "retinanet" if "retinanet" in cfg_path.lower() else "custommodel"
print(f"--- INFO: Model yang digunakan: {model_name} ---")


# ===== Step 2: Definisikan Pipeline Dasar =====
print("--- INFO: Mendefinisikan pipeline dasar untuk training dan validasi ---")
train_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=False),
    dict(type='Resize', scale=(416, 416), keep_ratio=True),
    dict(type='Pad', size=(416, 416), pad_val=dict(img=(114, 114, 114))),
]
val_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(type='Resize', scale=(416, 416), keep_ratio=True),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=False),
    dict(type='PackDetInputs',
         meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape', 'scale_factor'))
]
print("--- INFO: Definisi pipeline selesai. Dataset akan dibangun di Step 3. ---")

# ===== Step 3: Dataset & Kelas (Dengan Wrapper Mosaic) =====
# --- Diperbaiki untuk melatih HANYA pada 10 kelas yang diinginkan dari 12 kelas di dataset ---

# 1. Definisikan 10 kelas yang ingin Anda gunakan. Urutan di sini akan menentukan label model (0-9).
CLASSES = (
    'Betta', 'Discuss', 'Glofish', 'Goldfish', 'Guppy', 'Gurami',
    'Manfish', 'Molly', 'Swordtail', 'Tiger Barb'
)

# 2. Buat pemetaan dari ID asli di file JSON ke label baru (0-9).
# PENTING: Kita sengaja TIDAK menyertakan ID 2 (Corydoras) dan ID 8 (Janitor).
cat_id_map = {
    1: 0,  # Betta
    3: 1,  # Discuss
    4: 2,  # Glofish
    5: 3,  # Goldfish
    6: 4,  # Guppy
    7: 5,  # Gurami
    9: 6,  # Manfish
    10: 7, # Molly
    11: 8, # Swordtail
    12: 9  # Tiger Barb
}

# 3. Palet warna Anda yang berisi 10 warna sekarang sudah pas.
PALETTE = [
    (220, 20, 60), (0, 128, 255), (60, 179, 113), (255, 140, 0), (138, 43, 226),
    (255, 215, 0), (199, 21, 133), (127, 255, 212), (70, 130, 180), (255, 99, 71)
]

# Metainfo akan secara otomatis menggunakan variabel yang sudah diperbarui di atas.
metainfo = {'classes': CLASSES, 'palette': PALETTE, 'cat2label': cat_id_map}

print(f"--- INFO: Konfigurasi diatur untuk melatih {len(CLASSES)} kelas, mengabaikan kelas lain dari dataset. ---")

data_root = 'dataset/10_jenis_ikan/'
train_ann_file = 'train/_annotations.coco.json'
val_ann_file   = 'valid/_annotations.coco.json'
print(f"--- INFO: Anotasi train: {os.path.join(data_root, train_ann_file)}")
print(f"--- INFO: Anotasi val  : {os.path.join(data_root, val_ann_file)}")

train_dataset_cfg = dict(
    type='CocoDataset', data_root=data_root, metainfo=metainfo,
    ann_file=train_ann_file, data_prefix=dict(img='train/'),
    filter_cfg=dict(filter_empty_gt=True), pipeline=train_pipeline
)
cfg.train_dataloader.dataset = dict(
    type='MultiImageMixDataset', dataset=train_dataset_cfg,
    pipeline=[
        dict(type='PhotoMetricDistortion'),
        dict(type='RandomFlip', prob=0.5),
        dict(type='Normalize', mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True),
        dict(type='PackDetInputs')
    ]
)
# ========== PERUBAHAN: Batch Size 4 ==========
cfg.train_dataloader.batch_size = 4
cfg.val_dataloader.batch_size = 4
cfg.test_dataloader.batch_size = 4
print(f"--- INFO: Batch size diatur ke {cfg.train_dataloader.batch_size} untuk semua dataloader ---")

# Menambahkan num_workers untuk mempercepat data loading
cfg.train_dataloader.num_workers = 4 # Atur sesuai CPU Anda (misal: 2 atau 4)
cfg.val_dataloader.num_workers = 4
cfg.test_dataloader.num_workers = 4
print(f"--- INFO: Dataloader num_workers diatur ke {cfg.train_dataloader.num_workers} ---")


cfg.val_dataloader.dataset.update(dict(
    data_root=data_root, metainfo=metainfo, ann_file=val_ann_file,
    data_prefix=dict(img='valid/'), filter_cfg=dict(filter_empty_gt=True), pipeline=val_pipeline
))
cfg.test_dataloader.dataset.update(dict(
    data_root=data_root, metainfo=metainfo, ann_file=val_ann_file,
    data_prefix=dict(img='valid/'), filter_cfg=dict(filter_empty_gt=False), pipeline=val_pipeline
))

# ===== Step 4: Evaluator COCO (bbox) + classwise =====
cfg.val_evaluator.ann_file = os.path.join(data_root, val_ann_file)
cfg.test_evaluator.ann_file = os.path.join(data_root, val_ann_file)
cfg.val_evaluator.metric = 'bbox'
cfg.test_evaluator.metric = 'bbox'
cfg.val_evaluator.classwise = True
cfg.test_evaluator.classwise = True
os.makedirs('work_eval', exist_ok=True)
cfg.val_evaluator.outfile_prefix = os.path.join('work_eval', 'val_results')
cfg.test_evaluator.outfile_prefix = os.path.join('work_eval', 'test_results')


# ===== Step 5: Atur jumlah kelas di head =====
if hasattr(cfg.model, 'bbox_head'):
    cfg.model.bbox_head.num_classes = len(CLASSES)
elif hasattr(cfg.model, 'roi_head'):
    if isinstance(cfg.model.roi_head.bbox_head, list):
        for head in cfg.model.roi_head.bbox_head:
            head.num_classes = len(CLASSES)
    else:
        cfg.model.roi_head.bbox_head.num_classes = len(CLASSES)

# ===== Step 6: Training Settings =====

# --- BAGIAN 1: DEFINISI OPTIMIZER DAN LEARNING RATE STATIS ---
# ========== PERUBAHAN: Learning Rate STATIS 0.00001 ==========
cfg.optim_wrapper = dict(
    type='OptimWrapper',
    optimizer=dict(type='AdamW', lr=0.00001, weight_decay=0.05),
    paramwise_cfg=dict(
        custom_keys={
            'absolute_pos_embed': dict(decay_mult=0.),
            'relative_position_bias_table': dict(decay_mult=0.),
            'norm': dict(decay_mult=0.)
        }))

cfg.param_scheduler = []
print(f"--- INFO: Menggunakan Optimizer AdamW dengan Learning Rate STATIS {cfg.optim_wrapper['optimizer']['lr']} ---")
print("--- INFO: Learning rate scheduler dinonaktifkan. ---")


# --- BAGIAN 2: PENGATURAN PEMUATAN BOBOT ---
best_checkpoint_path = './outputs_swin_retinanet_10fish_finetune_color/best_coco_bbox_mAP_epoch_44.pth'
if not DEBUG_MODE and os.path.exists(best_checkpoint_path):
    cfg.load_from = best_checkpoint_path
    cfg.resume = False
    print(f"--- INFO: Memulai fine-tuning, memuat bobot dari: {cfg.load_from} ---")
else:
    cfg.load_from = None
    cfg.resume = False
    print(f"--- PERINGATAN: Checkpoint tidak ditemukan di '{best_checkpoint_path}'. Memulai dari awal. ---")

# --- BAGIAN 3: PENGATURAN EPOCH & HOOKS ---
# ========== PERUBAHAN: Epoch 30 ==========
if DEBUG_MODE:
    cfg.train_cfg.max_epochs = 2
else:
    cfg.train_cfg.max_epochs = 30

cfg.default_hooks.logger.interval = 10
cfg.default_hooks.checkpoint.interval = 1
cfg.default_hooks.checkpoint.max_keep_ckpts = 3
cfg.default_hooks.checkpoint.save_best = 'coco/bbox_mAP'
cfg.default_hooks.checkpoint.rule = 'greater'
cfg.visualizer.vis_backends = [
    dict(type='LocalVisBackend'),
    dict(type='TensorboardVisBackend')
]
print(f"--- INFO: Training akan berjalan selama {cfg.train_cfg.max_epochs} epoch. ---")


# ===== Step 7: Work dir =====
work_dir_base = './outputs_bs4_lr00001_epoch30'
cfg.work_dir = f"{work_dir_base}_debug" if DEBUG_MODE else work_dir_base
print(f"--- INFO: Output ke: {cfg.work_dir} ---")
os.makedirs(cfg.work_dir, exist_ok=True)


# ===== Step 8: Sanity check dataset =====
print("\n--- Pengecekan Dataset Final ---")
try:
    train_dataset = DATASETS.build(cfg.train_dataloader.dataset)
    print(f"Jumlah data training: {len(train_dataset)}")
    val_dataset = DATASETS.build(cfg.val_dataloader.dataset)
    print(f"Jumlah data validasi: {len(val_dataset)}")
except Exception as e:
    print(f"\nFATAL ERROR memuat dataset: {e}")
    raise e
print("---------------------------------\n")

print("Konfigurasi selesai. Siap training.")


# ===== Step 9: Training =====
print(">>> Mulai training...")
runner = Runner.from_cfg(cfg)
runner.train()
print(">>> Training selesai.")

# ========== STEP 10: EVALUATION & REPORTING ==========
print(">>> Evaluasi (COCO mAP + class-wise)...")
eval_results = runner.test()
if isinstance(eval_results, (list, tuple)) and len(eval_results) > 0:
    eval_dict = eval_results[0]
else:
    eval_dict = eval_results

report_dir = os.path.join(cfg.work_dir, 'reports')
os.makedirs(report_dir, exist_ok=True)
eval_txt = os.path.join(report_dir, 'coco_eval_summary.txt')
with open(eval_txt, 'w', encoding='utf-8') as f:
    f.write("=== COCO BBox Evaluation (mAP) ===\n")
    for k, v in eval_dict.items():
        f.write(f"{k}: {v}\n")
print(f">>> Ringkasan evaluasi disimpan di: {eval_txt}")

# Evaluator Custom Functions
def compute_iou_matrix(boxes1, boxes2):
    if boxes1.size == 0 or boxes2.size == 0: return np.zeros((boxes1.shape[0], boxes2.shape[0]), dtype=np.float32)
    x11, y11, x12, y12 = boxes1[:,0], boxes1[:,1], boxes1[:,2], boxes1[:,3]
    x21, y21, x22, y22 = boxes2[:,0], boxes2[:,1], boxes2[:,2], boxes2[:,3]
    inter_x1 = np.maximum(x11[:, None], x21[None, :]); inter_y1 = np.maximum(y11[:, None], y21[None, :])
    inter_x2 = np.minimum(x12[:, None], x22[None, :]); inter_y2 = np.minimum(y12[:, None], y22[None, :])
    inter_w = np.maximum(0, inter_x2 - inter_x1); inter_h = np.maximum(0, inter_y2 - inter_y1)
    inter = inter_w * inter_h
    area1 = (x12 - x11) * (y12 - y11); area2 = (x22 - x21) * (y22 - y21)
    union = area1[:, None] + area2[None, :] - inter
    return np.where(union > 0, inter / union, 0.0)

def greedy_match(iou_mat, iou_thr=0.5):
    matches = []; gt_used, pred_used = set(), set()
    if iou_mat.size == 0: return matches
    pairs = sorted([(i, j, iou_mat[i, j]) for i in range(iou_mat.shape[0]) for j in range(iou_mat.shape[1])], key=lambda x: x[2], reverse=True)
    for i, j, iou in pairs:
        if iou < iou_thr: break
        if i in gt_used or j in pred_used: continue
        gt_used.add(i); pred_used.add(j); matches.append((i, j))
    return matches

def evaluate_confusion_pr(model, dataset, score_thr=0.3, iou_thr=0.5):
    base_dataset = dataset.dataset if hasattr(dataset, 'dataset') else dataset
    n_classes = len(CLASSES)
    conf_mat = np.zeros((n_classes, n_classes), dtype=np.int32)
    per_class_counts = {'TP': np.zeros(n_classes, dtype=np.int32), 'FP': np.zeros(n_classes, dtype=np.int32), 'FN': np.zeros(n_classes, dtype=np.int32)}
    pr_store = {c: {'scores': [], 'match': []} for c in range(n_classes)}

    for idx in range(len(base_dataset)):
        info = base_dataset.get_data_info(idx)
        img_path = info['img_path']

        if 'instances' in info:
            gt_bboxes = np.array([ann['bbox'] for ann in info['instances']], dtype=np.float32).reshape(-1, 4)
            gt_labels = np.array([ann['bbox_label'] for ann in info['instances']], dtype=np.int64)
        else: gt_bboxes = np.zeros((0, 4), dtype=np.float32); gt_labels = np.array([], dtype=np.int64)

        result = inference_detector(model, img_path)
        pred = result.pred_instances
        pred_bboxes, pred_scores, pred_labels = pred.bboxes.cpu().numpy(), pred.scores.cpu().numpy(), pred.labels.cpu().numpy()

        keep = pred_scores >= score_thr
        pred_bboxes_k, pred_scores_k, pred_labels_k = pred_bboxes[keep], pred_scores[keep], pred_labels[keep]

        iou_mat = compute_iou_matrix(gt_bboxes, pred_bboxes_k)
        matches = greedy_match(iou_mat, iou_thr=iou_thr)
        matched_gt, matched_pred = {m[0] for m in matches}, {m[1] for m in matches}

        for gi, pj in matches:
            gt_c, pd_c = int(gt_labels[gi]), int(pred_labels_k[pj])
            conf_mat[gt_c, pd_c] += 1; per_class_counts['TP'][pd_c] += 1
            pr_store[pd_c]['scores'].append(float(pred_scores_k[pj])); pr_store[pd_c]['match'].append(1 if gt_c == pd_c else 0)
        for j in range(len(pred_bboxes_k)):
            if j not in matched_pred:
                pd_c = int(pred_labels_k[j]); per_class_counts['FP'][pd_c] += 1
                pr_store[pd_c]['scores'].append(float(pred_scores_k[j])); pr_store[pd_c]['match'].append(0)
        for i in range(len(gt_bboxes)):
            if i not in matched_gt: per_class_counts['FN'][int(gt_labels[i])] += 1

    eps = 1e-12
    prec = per_class_counts['TP'] / (per_class_counts['TP'] + per_class_counts['FP'] + eps)
    rec  = per_class_counts['TP'] / (per_class_counts['TP'] + per_class_counts['FN'] + eps)
    f1   = 2 * prec * rec / (prec + rec + eps)

    lines = [f"{name}, {per_class_counts['TP'][c]}, {per_class_counts['FP'][c]}, {per_class_counts['FN'][c]}, {prec[c]:.4f}, {rec[c]:.4f}, {f1[c]:.4f}" for c, name in enumerate(CLASSES)]
    lines.insert(0, "KELAS, TP, FP, FN, Precision, Recall, F1")
    per_class_txt = os.path.join(report_dir, "per_class_metrics.txt")
    with open(per_class_txt, "w", encoding="utf-8") as f: f.write("\n".join(lines))

    fig_cm = plt.figure(figsize=(10, 8)); ax = fig_cm.add_subplot(111); im = ax.imshow(conf_mat, interpolation='nearest')
    ax.set_title("Confusion Matrix (IoU>0.5)"); plt.colorbar(im, ax=ax); tick_marks = np.arange(len(CLASSES))
    ax.set_xticks(tick_marks, labels=CLASSES, rotation=45, ha='right'); ax.set_yticks(tick_marks, labels=CLASSES)
    thresh = conf_mat.max() * 0.6 if conf_mat.max() > 0 else 0.5
    for i, j in itertools.product(range(conf_mat.shape[0]), range(conf_mat.shape[1])): ax.text(j, i, f"{conf_mat[i, j]}", ha="center", va="center", color="white" if conf_mat[i, j] > thresh else "black", fontsize=8)
    plt.tight_layout(); cm_path = os.path.join(report_dir, "confusion_matrix.png"); fig_cm.savefig(cm_path, dpi=200); plt.close(fig_cm)

    for c in range(len(CLASSES)):
        scores, match = np.array(pr_store[c]['scores']), np.array(pr_store[c]['match'])
        if scores.size == 0: continue
        order = np.argsort(-scores); scores, match = scores[order], match[order]
        tp_cum, fp_cum = np.cumsum(match), np.cumsum(1 - match)
        precision = tp_cum / np.maximum(tp_cum + fp_cum, 1); total_pos = per_class_counts['TP'][c] + per_class_counts['FN'][c]
        recall = tp_cum / max(total_pos, 1); fig_pr = plt.figure()
        plt.plot(recall, precision); plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(f"PR Curve - {CLASSES[c]}")
        pr_path = os.path.join(report_dir, f"pr_curve_{c:02d}_{CLASSES[c].replace(' ','_')}.png"); fig_pr.savefig(pr_path, dpi=200, bbox_inches='tight'); plt.close(fig_pr)

    print(f">>> Confusion matrix: {cm_path}"); print(f">>> Per-class metrics: {per_class_txt}"); print(f">>> PR curves disimpan di: {report_dir}")

# ========== Load Model untuk Evaluasi ==========
best_checkpoint_name = None
if os.path.exists(cfg.work_dir):
    best_ckpts = [f for f in os.listdir(cfg.work_dir) if f.startswith('best_') and f.endswith('.pth')]
    if best_ckpts: best_checkpoint_name = sorted(best_ckpts)[-1]
checkpoint_path = os.path.join(cfg.work_dir, best_checkpoint_name) if best_checkpoint_name else os.path.join(cfg.work_dir, "latest.pth")

if os.path.exists(checkpoint_path):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = init_detector(cfg, checkpoint_path, device=device)
    _ = evaluate_confusion_pr(model, val_dataset, score_thr=0.3, iou_thr=0.5)
else:
    print(f"PERINGATAN: Tidak ada checkpoint yang ditemukan di {checkpoint_path}. Melewatkan evaluasi custom.")

# ========== STEP 11: INFERENCE + RINGKASAN JUMLAH PER JENIS ==========
if os.path.exists(checkpoint_path):
    vis_save_dir = os.path.join(cfg.work_dir, 'vis_outputs')
    os.makedirs(vis_save_dir, exist_ok=True)
    visualizer = DetLocalVisualizer(vis_backends=[dict(type='LocalVisBackend')], name='visualizer', save_dir=vis_save_dir)
    visualizer.dataset_meta = cfg.train_dataloader.dataset.metainfo

    img_path_test = 'dataset/10jenisikantest/test/Manfish_120_jpg.rf.d6bdd77a6bf451129c68ddd311776eb4.jpg'
    if not os.path.exists(img_path_test):
        print(f"ERROR: Gambar uji tidak ditemukan di '{img_path_test}'")
    else:
        result = inference_detector(model, img_path_test)
        pred_instances = result.pred_instances
        scores, labels = pred_instances.scores.cpu().numpy(), pred_instances.labels.cpu().numpy()

        score_thr = 0.3; keep = scores >= score_thr; labels_k = labels[keep]
        counts = {name: int((labels_k == i).sum()) for i, name in enumerate(CLASSES)}
        total_detected = int(keep.sum())

        print(">>> Summary Deteksi per Jenis (score_thr=0.3):")
        for k, v in counts.items(): print(f"- {k}: {v}")
        print(f"Total semua ikan terdeteksi: {total_detected}")

        summary_txt = os.path.join(vis_save_dir, "summary_counts.txt")
        with open(summary_txt, "w", encoding="utf-8") as f:
            f.write(f"Summary Deteksi per Jenis (score_thr={score_thr})\n")
            for k, v in counts.items(): f.write(f"{k}: {v}\n")
            f.write(f"TOTAL: {total_detected}\n")
        print(f">>> Ringkasan disimpan: {summary_txt}")

        img = mmcv.imread(img_path_test); img = mmcv.imconvert(img, 'bgr', 'rgb')
        out_file = os.path.join(vis_save_dir, "deteksi_result.jpg")
        visualizer.add_datasample(name='result', image=img, data_sample=result, draw_gt=False, show=False, wait_time=0, pred_score_thr=score_thr, out_file=out_file)
        print(f">>> Gambar hasil deteksi disimpan di: {out_file}")
else:
    print(f"PERINGATAN: Tidak ada checkpoint. Melewatkan inferensi.")

In [ ]:
import os
import torch
from mmengine.config import Config
from mmengine.runner import Runner
from mmdet.utils import register_all_modules

print("--- Memulai Skrip Evaluasi Ulang (Versi Final dengan Perbaikan Model) ---")

# ==================== PENGATURAN ====================
cfg_path = 'mmdetection/configs/swin/retinanet_swin-s-p4-w7_fpn_1x_coco_custom.py'
checkpoint_path = 'c:/Users/Stevenstven/Documents/VSCode/Skripsi/Deteksi_Jenis_Ikan/outputs_model_dengan_grafik_epoch300_batchsize4_lr0.00001/best_coco_bbox_mAP_epoch_234.pth'
work_dir = './outputs_model_dengan_grafik_epoch300_batchsize4_lr0.00001'

data_root = 'dataset/10_jenis_ikan/'
ann_file = 'valid/_annotations.coco.json'
data_prefix = 'valid/'

CLASSES = (
    'Betta', 'Discuss', 'Glofish', 'Goldfish', 'Guppy', 'Gurami',
    'Manfish', 'Molly', 'Swordtail', 'Tiger Barb'
)

CAT_ID_MAP = {
    1: 0, 3: 1, 4: 2, 5: 3, 6: 4, 7: 5, 9: 6, 10: 7, 11: 8, 12: 9
}
# ======================================================

# Daftarkan semua modul MMDetection
register_all_modules(init_default_scope=False)

# Muat konfigurasi dasar dari file
cfg = Config.fromfile(cfg_path)


# --- BAGIAN PALING KRUSIAL: PERBAIKI SEMUA KONFIGURASI ---
print("--- INFO: Memperbaiki semua konfigurasi yang diperlukan...")

# 1. PERBAIKI JUMLAH KELAS DI MODEL (INI YANG PALING PENTING!)
cfg.model.bbox_head.num_classes = len(CLASSES)

# 2. Definisikan pipeline validasi
val_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(type='Resize', scale=(416, 416), keep_ratio=True),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='PackDetInputs',
        meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape', 'scale_factor'))
]

# 3. Bangun ulang Dataloader untuk Pengujian
cfg.test_dataloader = dict(
    batch_size=8,
    num_workers=2,
    persistent_workers=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type='CocoDataset',
        data_root=data_root,
        ann_file=ann_file,
        data_prefix=dict(img=data_prefix),
        test_mode=True,
        pipeline=val_pipeline,
        metainfo=dict(classes=CLASSES, cat2label=CAT_ID_MAP)
    )
)

# 4. Bangun ulang Evaluator untuk Pengujian
cfg.test_evaluator = dict(
    type='CocoMetric',
    ann_file=os.path.join(data_root, ann_file),
    metric='bbox',
    classwise=True,
)
# --- AKHIR DARI BAGIAN KRUSIAL ---


# Atur path checkpoint yang akan dievaluasi
cfg.load_from = checkpoint_path

# Atur direktori kerja
cfg.work_dir = work_dir
report_dir = os.path.join(work_dir, 'final_reports_LENGKAP')
os.makedirs(report_dir, exist_ok=True)

# Bangun runner dari konfigurasi yang sudah kita perbaiki total
runner = Runner.from_cfg(cfg)

# Jalankan HANYA proses evaluasi
print("\n>>> Menjalankan evaluasi COCO standar dengan metrik per kelas...")
eval_results = runner.test()

# Simpan ringkasan evaluasi yang LENGKAP ke file teks
eval_summary_path = os.path.join(report_dir, 'coco_eval_summary_LENGKAP.txt')
with open(eval_summary_path, 'w', encoding='utf-8') as f:
    f.write("=== COCO BBox Evaluation (mAP) - LENGKAP ===\n")
    for k, v in eval_results.items():
        if 'mAP' in k or 'precision' in k:
            f.write(f"{k}: {v:.4f}\n")

print(f"\n🎉 BERHASIL! Laporan lengkap disimpan di: {eval_summary_path}")
print("--- Skrip evaluasi ulang selesai. ---")

In [ ]:
# ===== Step 7: Work dir (dengan Timestamp Otomatis) =====
from datetime import datetime

# Buat timestamp unik untuk nama folder
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Ambil info dasar dari konfigurasi untuk penamaan folder
batch_size = cfg.train_dataloader.batch_size
learning_rate = cfg.optim_wrapper.optimizer.lr

# Definisikan nama folder yang deskriptif
work_dir_base = f'./outputs_{model_name}_bs{batch_size}_lr{learning_rate}_{timestamp}'

cfg.work_dir = f"{work_dir_base}_debug" if DEBUG_MODE else work_dir_base
print(f"--- INFO: Output akan disimpan di direktori unik: {cfg.work_dir} ---")
os.makedirs(cfg.work_dir, exist_ok=True)

In [ ]:
# ===== Step 9: Training =====
import time

print(">>> Mulai training...")
start_time = time.time()

runner = Runner.from_cfg(cfg)
runner.train()

end_time = time.time()
training_duration_seconds = end_time - start_time
print(f">>> Training selesai dalam {training_duration_seconds:.2f} detik.")

In [ ]:
# ========== STEP 10: EVALUASI & PELAPORAN KOMPREHENSIF ==========
import json
import matplotlib.pyplot as plt
import pandas as pd

# --- Bagian 1: Setup Direktori & Path ---
print("\n" + "="*50)
print("      MEMULAI PROSES EVALUASI & PELAPORAN")
print("="*50)

# Buat direktori khusus untuk semua laporan di dalam work_dir
report_dir = os.path.join(cfg.work_dir, f'evaluation_reports_{timestamp}')
os.makedirs(report_dir, exist_ok=True)
print(f"--- INFO: Semua file laporan akan disimpan di: {report_dir} ---")

# Cari checkpoint terbaik secara otomatis
best_checkpoint_path = None
if os.path.exists(cfg.work_dir):
    # Cari file .pth yang mengandung 'best' di namanya
    best_ckpts = [f for f in os.listdir(cfg.work_dir) if f.startswith('best_') and f.endswith('.pth')]
    if best_ckpts:
        best_checkpoint_path = os.path.join(cfg.work_dir, best_ckpts[0])
        print(f"--- INFO: Checkpoint terbaik ditemukan: {best_checkpoint_path} ---")
    else:
        # Jika tidak ada, pakai checkpoint terakhir
        latest_checkpoint_path = os.path.join(cfg.work_dir, "latest.pth")
        if os.path.exists(latest_checkpoint_path):
            best_checkpoint_path = latest_checkpoint_path
            print(f"--- PERINGATAN: Checkpoint terbaik tidak ditemukan. Menggunakan checkpoint terakhir: {best_checkpoint_path} ---")

if not best_checkpoint_path or not os.path.exists(best_checkpoint_path):
    raise FileNotFoundError("Tidak ada file checkpoint yang bisa dievaluasi. Hentikan proses.")


# --- Bagian 2: Parsing Log Training untuk Grafik ---
def parse_and_plot_logs(work_dir, save_dir):
    """Membaca file log JSON dari MMDetection, mengekstrak metrik, dan membuat plot."""
    log_files = [f for f in os.listdir(os.path.join(work_dir, 'vis_data')) if f.endswith('.json')]
    if not log_files:
        print("--- PERINGATAN: File log JSON tidak ditemukan. Grafik tidak dapat dibuat. ---")
        return None, None

    log_path = os.path.join(work_dir, 'vis_data', log_files[0])
    
    epochs = []
    train_losses = []
    val_maps = []
    best_map = {"map": 0.0, "epoch": 0}

    with open(log_path, 'r') as f:
        for line in f:
            log_data = json.loads(line.strip())
            epoch = log_data.get('epoch', log_data.get('step', 0)) # Ambil epoch atau step
            
            # Ambil training loss
            if 'loss' in log_data:
                epochs.append(epoch)
                train_losses.append(log_data['loss'])
            
            # Ambil validation mAP dan catat yang terbaik
            if 'coco/bbox_mAP' in log_data:
                current_map = log_data['coco/bbox_mAP']
                val_maps.append({'epoch': epoch, 'map': current_map})
                if current_map > best_map['map']:
                    best_map['map'] = current_map
                    best_map['epoch'] = epoch

    # Plotting Training Loss
    plt.figure(figsize=(12, 6))
    plt.plot(epochs, train_losses, label='Training Loss')
    plt.title('Training Loss per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    loss_curve_path = os.path.join(save_dir, 'training_loss_curve.png')
    plt.savefig(loss_curve_path)
    plt.close()
    print(f">>> Grafik kurva loss disimpan di: {loss_curve_path}")

    # Plotting Validation mAP
    if val_maps:
        val_df = pd.DataFrame(val_maps)
        plt.figure(figsize=(12, 6))
        plt.plot(val_df['epoch'], val_df['map'], marker='o', linestyle='-', label='Validation mAP')
        plt.title('Validation mAP per Epoch')
        plt.xlabel('Epoch')
        plt.ylabel('COCO bbox_mAP')
        # Tandai titik mAP terbaik
        if best_map['epoch'] > 0:
            plt.scatter(best_map['epoch'], best_map['map'], color='red', zorder=5, label=f"Best mAP: {best_map['map']:.4f} at Epoch {best_map['epoch']}")
        plt.legend()
        plt.grid(True)
        map_curve_path = os.path.join(save_dir, 'validation_mAP_curve.png')
        plt.savefig(map_curve_path)
        plt.close()
        print(f">>> Grafik kurva mAP disimpan di: {map_curve_path}")

    return best_map, (loss_curve_path, map_curve_path if val_maps else None)

best_map_info, (loss_plot, map_plot) = parse_and_plot_logs(cfg.work_dir, report_dir)


# --- Bagian 3: Evaluasi Model Standar (COCO mAP) ---
print("\n>>> Memulai evaluasi standar (COCO mAP + class-wise)...")
eval_results = runner.test()
eval_dict = eval_results[0] if isinstance(eval_results, (list, tuple)) else eval_results

# Simpan ringkasan evaluasi COCO ke file
eval_txt_path = os.path.join(report_dir, 'coco_eval_summary.txt')
with open(eval_txt_path, 'w', encoding='utf-8') as f:
    f.write("=== COCO BBox Evaluation (mAP) ===\n")
    for k, v in eval_dict.items():
        f.write(f"{k}: {v}\n")
print(f">>> Ringkasan evaluasi COCO disimpan di: {eval_txt_path}")


# --- Bagian 4: Evaluasi Kustom (Confusion Matrix & PR Curves) ---
print("\n>>> Memulai evaluasi kustom (Confusion Matrix, Per-Class Metrics, PR Curves)...")

# Pastikan torch.load di-patch lagi jika kernel di-restart
_orig_torch_load = torch.load
def torch_load_wrapper(*args, **kwargs):
    kwargs["weights_only"] = False
    return _orig_torch_load(*args, **kwargs)
torch.load = torch_load_wrapper

# Inisialisasi model dari checkpoint terbaik
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = init_detector(cfg, best_checkpoint_path, device=device)

# Panggil fungsi evaluasi kustom Anda, pastikan output disimpan ke direktori laporan yang benar
# (Fungsi 'evaluate_confusion_pr' dari sel Anda sebelumnya diasumsikan ada di sini)
custom_eval_results = evaluate_confusion_pr(model, val_dataset, score_thr=0.3, iou_thr=0.5)
print(">>> Evaluasi kustom selesai.")


# --- Bagian 5: Membuat Laporan Ringkasan Akhir ---
summary_report_path = os.path.join(report_dir, 'final_training_summary.txt')
with open(summary_report_path, 'w', encoding='utf-8') as f:
    f.write("="*50 + "\n")
    f.write("      RINGKASAN SESI TRAINING & EVALUASI\n")
    f.write("="*50 + "\n\n")

    f.write(f"Timestamp Sesi: {timestamp}\n")
    f.write(f"Direktori Output: {cfg.work_dir}\n")
    f.write(f"Direktori Laporan: {report_dir}\n\n")

    f.write("--- Konfigurasi Training ---\n")
    f.write(f"Model: {model_name}\n")
    f.write(f"Checkpoint Terbaik: {os.path.basename(best_checkpoint_path)}\n")
    f.write(f"Learning Rate: {learning_rate}\n")
    f.write(f"Batch Size: {batch_size}\n")
    f.write(f"Total Epoch: {cfg.train_cfg.max_epochs}\n")
    f.write(f"Perangkat Keras: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}\n\n")

    m, s = divmod(training_duration_seconds, 60)
    h, m = divmod(m, 60)
    f.write(f"--- Durasi & Performa ---\n")
    f.write(f"Total Waktu Training: {int(h)} jam, {int(m)} menit, {s:.2f} detik\n")
    if best_map_info:
        f.write(f"mAP Terbaik (coco/bbox_mAP): {best_map_info['map']:.4f} (dicapai pada epoch {best_map_info['epoch']})\n\n")
    
    f.write("--- Hasil Evaluasi ---\n")
    f.write(f"Ringkasan Metrik COCO: Lihat '{os.path.basename(eval_txt_path)}'\n")
    f.write(f"Metrik Per Kelas (P/R/F1): Lihat '{os.path.basename(custom_eval_results['paths']['txt'])}'\n\n")

    f.write("--- Lokasi File Grafik ---\n")
    if loss_plot:
        f.write(f"Grafik Training Loss: '{os.path.basename(loss_plot)}'\n")
    if map_plot:
        f.write(f"Grafik Validation mAP: '{os.path.basename(map_plot)}'\n")
    f.write(f"Gambar Confusion Matrix: '{os.path.basename(custom_eval_results['paths']['cm'])}'\n")
    f.write(f"Direktori Kurva PR: Sub-direktori di dalam laporan\n")

print("\n" + "="*50)
print(f"      PROSES PELAPORAN SELESAI")
print(f">>> Laporan ringkasan akhir disimpan di: {summary_report_path}")
print("="*50)